In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tues Nov  6 10:08:33 2018

@author: Anum Hassan
"""
#importing libraries
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.layers import MaxPooling2D, ZeroPadding2D
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Activation
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')


seed = 7
np.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')


# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]




# create model
model = Sequential()
convolution1=Convolution2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu')
model.add(convolution1)
pooling1=MaxPooling2D(pool_size=(2, 2))
model.add(pooling1)
convolution2=Convolution2D(15, (3, 3), activation='relu')
model.add(convolution2)
pooling2=(MaxPooling2D(pool_size=(2, 2)))
model.add(pooling2)
convolution3=(Convolution2D(7, (2, 2), activation='relu'))
model.add(convolution3)
pooling3=(MaxPooling2D(pool_size=(2, 2)))
model.add(pooling3)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()



# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Fit the model
history =model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))


# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# choose any image to want by specifying the index
img_to_visualize = X_train[65]
# Keras requires the image to be in 4D
# So we add an extra dimension to it.
img_to_visualize = np.expand_dims(img_to_visualize, axis=0)

def layer_to_visualize(layer):
    inputs = [K.learning_phase()] + model.inputs

    _convout1_f = K.function(inputs, [layer.output])
    def convout1_f(X):
        # The [0] is to disable the training phase flag
        return _convout1_f([0] + [X])

    convolutions = convout1_f(img_to_visualize)
    convolutions = np.squeeze(convolutions)

    print ('Shape of conv:', convolutions.shape)
    
    n = convolutions.shape[0]
    n = int(np.ceil(np.sqrt(n)))
    
    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(12,8))
    for i in range(len(convolutions)):
        ax = fig.add_subplot(n,n,i+1)
        ax.imshow(convolutions[i], cmap='gray')
        
        
        
        
        # Specify the layer to want to visualize
layer_to_visualize(convolution1)

layer_to_visualize(convolution2)

layer_to_visualize(convolution3) 







Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 24, 24)        780       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 12, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 10, 10)        4065      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 5, 5)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 4, 4)           427       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 2, 2)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 2, 2)           0         
__________

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Shape of conv: (30, 24, 24)
Shape of conv: (15, 10, 10)
Shape of conv: (7, 4, 4)


In [2]:
#get weights
l1_weights=model.layers[8].get_weights()[0]
print(l1_weights)
l1_biases=model.layers[8].get_weights()[1]
print(l1_biases)
l2_weights=model.layers[9].get_weights()[0]
print(l2_weights)
l2_biases=model.layers[9].get_weights()[1]
print(l2_biases)



[[-0.02679238  0.08436859  0.16123211 ...  0.05955197 -0.12143186
   0.21741721]
 [-0.02794815  0.2589801   0.03387017 ...  0.02344163 -0.18102162
   0.11701164]
 [ 0.00940209  0.07822873 -0.0468776  ...  0.02596794 -0.03961542
   0.14156052]
 ...
 [ 0.09385799 -0.12669414 -0.10353576 ...  0.24891116  0.09361523
   0.04917668]
 [ 0.31748024  0.13822773  0.2795041  ...  0.13573906 -0.1900266
   0.08564367]
 [-0.20463865 -0.29737467 -0.06173779 ... -0.06571663 -0.04734336
   0.0805827 ]]
[-0.02036037  0.11338935  0.23903324 -0.0152906   0.08479903 -0.04510693
 -0.05521713  0.04450922  0.01431249 -0.06937889 -0.1269956  -0.05209601
  0.17097104  0.1720261  -0.01189715  0.14187822  0.05058365  0.19899806
  0.17538227  0.0296684   0.13129716  0.26089314  0.07793213  0.15284882
  0.3002597  -0.11122948 -0.08463863  0.02899291 -0.01687285  0.16679606
  0.21232782  0.05604663  0.38832536 -0.02749468  0.08198337 -0.12388838
  0.09728178  0.08422448  0.15083182  0.03673638  0.13982551  0.1345115